In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import numpy as np

# = = = = = = = = = = = = = = =

import json

In [2]:
directory_in = './file/input'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'fuck_new.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：753



,No,Part Number
0,1,RD 10160C
1,3,RD 1789SC
2,4,HA 10954C
3,5,RD 2203SC
4,6,RD 2151C
...,...,...
748,1737,EX 10747
749,1738,EX 10561C
750,1739,EX 529442C
751,1742,SW 2155C


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                     1
 Part Number    RD 10160C
 Name: 0, dtype: object)

In [5]:
import time

data = {'search': crawler_series['Part Number'],
        'page': '1'}

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_check = f'''https://uacparts.com/Catalog/SearchIndex?search={crawler_series['Part Number']}'''

        resp = requests.post('https://uacparts.com/SearchAjaxList/Search',
                             data=data,
                             headers=get_header(),
                             proxies=get_proxy(),
                             timeout=(5, 15))

        if resp.status_code == 200:
            dict_ = resp.json()

            if dict_['d']['it'] == None and (dict_['d']['m'] == False or dict_['d']['m'] == True) and dict_['d']['th'] == None and dict_['d']['p'] == None:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_

1


{'d': {'it': None,
  'm': False,
  'c': '\r<li id=\'part32400\' class="awe-li part awe-il" data-val="32400"><div class="label-bar">\r<h2 class="part-label">RD 10160</h2></div><div>\r<div class="part-image">\r<button class="btnleft" style="padding: 2px 40px;position:relative;top:50%;left:0%;color:white;background:#003366;" onclick="slide(-1,32400)">&#10094;</button>\r<button class="btnright" style="padding: 2px 40px;position:relative;top:50%;right:0%;color:white;background:#003366;" onclick="slide(+1,32400)">&#10095;</button>\r<img src=\'https://www.uacparts.com/photo/WEBP/RD/RD 10160_1_730.webp\' style="display:block;" onclick="LoadImage(32400)" class="thumb thumb-part mySlides_32400" onerror="this.onerror=null;this.src=\'/images/nophoto.jpg\'" alt=\'picture\' />\r<img src=\'https://www.uacparts.com/photo/WEBP/RD/RD 10160_3_730.webp\' style="display:none;" onclick="LoadImage(32400)" class="thumb thumb-part mySlides_32400" onerror="this.onerror=null;this.src=\'/images/nophoto.jpg\'" alt

In [6]:
soup = BeautifulSoup(dict_['d']['c'], 'lxml')
html = etree.HTML(str(soup))

with open('./part.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <li class="awe-li part awe-il" data-val="32400" id="part32400">
   <div class="label-bar">
    <h2 class="part-label">
     RD 10160
    </h2>
   </div>
   <div>
    <div class="part-image">
     <button class="btnleft" onclick="slide(-1,32400)" style="padding: 2px 40px;position:relative;top:50%;left:0%;color:white;background:#003366;">
      ❮
     </button>
     <button class="btnright" onclick="slide(+1,32400)" style="padding: 2px 40px;position:relative;top:50%;right:0%;color:white;background:#003366;">
      ❯
     </button>
     <img alt="picture" class="thumb thumb-part mySlides_32400" onclick="LoadImage(32400)" onerror="this.onerror=null;this.src='/images/nophoto.jpg'" src="https://www.uacparts.com/photo/WEBP/RD/RD 10160_1_730.webp" style="display:block;"/>
     <img alt="picture" class="thumb thumb-part mySlides_32400" onclick="LoadImage(32400)" onerror="this.onerror=null;this.src='/images/nophoto.jpg'" src="https://www.uacparts.com/photo/WEBP/RD/RD 10160_3_730

In [7]:
list_li = html.xpath('//li[@class="awe-li part awe-il"]')

list_li

[<Element li at 0x11d8385c0>, <Element li at 0x11d7ed580>]

In [8]:
list_part_number = [li.xpath('./div[@class="label-bar"]/h2[@class="part-label"]/text()')[0] for li in list_li]

list_part_number

['RD 10160', 'RD 10160C']

In [9]:
array_index_part = np.where(np.array([part_number.replace(' ', '') for part_number in list_part_number]) == crawler_series['Part Number'].replace(' ', ''))[0]

array_index_part

array([1])

In [10]:
if array_index_part.size == 1:
    index_part = array_index_part[0]

index_part

np.int64(1)

In [11]:
dict_src, list_src = {}, [href.strip() for href in list_li[index_part].xpath('./div/div[@class="part-image"]/img[@alt="picture"]/@src')]
for i, src in enumerate(list_src):
    dict_src[str(i)] = src

dict_src

{'0': 'https://www.uacparts.com/photo/WEBP/RD/RD 10160C_1_743.webp',
 '1': 'https://www.uacparts.com/photo/WEBP/RD/RD 10160C_2_743.webp'}

In [12]:
dict_note, list_text = {}, list_li[index_part].xpath('./div/div[@class="part-info"]/div[@class="comments"]/text()')
for i, text in enumerate(list_text):
    dict_note[str(i)] = text

dict_note

{'0': 'Accumulator'}

In [13]:
list_dict_correct.append({'Part Number': crawler_series['Part Number'],
                          '[UAC] Part Number': list_part_number[index_part],
                          '[UAC] Picture': '',
                          '[UAC] Url': url_check,
                          '[UAC] Json_Src': json.dumps(dict_src),
                          '[UAC] Json_Note': json.dumps(dict_note)})

list_dict_correct

[{'Part Number': 'RD 10160C',
  '[UAC] Part Number': 'RD 10160C',
  '[UAC] Picture': '',
  '[UAC] Url': 'https://uacparts.com/Catalog/SearchIndex?search=RD 10160C',
  '[UAC] Json_Src': '{"0": "https://www.uacparts.com/photo/WEBP/RD/RD 10160C_1_743.webp", "1": "https://www.uacparts.com/photo/WEBP/RD/RD 10160C_2_743.webp"}',
  '[UAC] Json_Note': '{"0": "Accumulator"}'}]

In [14]:
crawler_status = True

crawler_status

True

In [15]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Part Number                                            RD 10160C
 Check_Url      https://uacparts.com/Catalog/SearchIndex?searc...
 Name: 0, dtype: object]

In [16]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Part Number'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_part.xlsx', index=False)

df_correct

,Part Number,[UAC] Part Number,[UAC] Picture,[UAC] Url,[UAC] Json_Src,[UAC] Json_Note
0,RD 10160C,RD 10160C,,https://uacparts.com/Catalog/SearchIndex?searc...,"{""0"": ""https://www.uacparts.com/photo/WEBP/RD/...","{""0"": ""Accumulator""}"


In [17]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Part Number'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_error.xlsx', index=False)

df_error

,No,Part Number,Check_Url
0,1,RD 10160C,https://uacparts.com/Catalog/SearchIndex?searc...
